In [1]:

from google.colab import drive
import os

# Mount Drive (để lấy Model PhoBERT xịn)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
!pip install streamlit transformers torch vncorenlp -q

# Tải công cụ Cloudflare (Thay thế Ngrok)
!wget -q -O cloudflared-linux-amd64 https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

# Tải VnCoreNLP (Bắt buộc để chạy app)
!mkdir -p vncorenlp/models/wordsegmenter
!wget -q -P vncorenlp/ https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget -q -P vncorenlp/models/wordsegmenter/ https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget -q -P vncorenlp/models/wordsegmenter/ https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 122.0 MB/s eta 0:00:00


In [2]:
app_code = """
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from vncorenlp import VnCoreNLP
import re

# CẤU HÌNH ĐƯỜNG DẪN
# Bạn kiểm tra kỹ đường dẫn này trong Drive của bạn nhé
MODEL_PATH = "/content/drive/My Drive/NLP Project/Model_PhoBERT"
VNCORENLP_PATH = "vncorenlp/VnCoreNLP-1.1.1.jar"

id2label = {
    0: "THỂ THAO ⚽", 1: "SỨC KHỎE 🏥", 2: "GIÁO DỤC 🎓",
    3: "PHÁP LUẬT ⚖️", 4: "KINH DOANH 💰", 5: "THƯ GIÃN 😂", 6: "KHOA HỌC CÔNG NGHỆ 🖥️", 7: "XE CỘ 🏎️",
    8: "ĐỜI SỐNG 💁‍♂️", 9: "THẾ GIỚI 🌍"
}

@st.cache_resource
def load_resources():
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
        model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
        rdrsegmenter = VnCoreNLP(VNCORENLP_PATH, annotators="wseg", max_heap_size='-Xmx500m')
        return tokenizer, model, rdrsegmenter
    except Exception as e:
        return None, None, None

tokenizer, model, rdrsegmenter = load_resources()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^\w\sàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ0-9.,?!]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def preprocess(text):
    text = clean_text(text)
    try:
        sentences = rdrsegmenter.tokenize(text)
        return " ".join([" ".join(sentence) for sentence in sentences])
    except:
        return text

st.title("📰 PHÂN LOẠI TIN TỨC (PhoBERT)")
st.write("Dán nội dung bài báo vào đây:")
input_text = st.text_area("", height=150)

if st.button("Phân loại"):
    if not tokenizer:
        st.error(f"⚠️ Không tìm thấy Model tại: {MODEL_PATH}. Hãy kiểm tra lại Drive!")
    elif not input_text.strip():
        st.warning("Chưa nhập nội dung!")
    else:
        with st.spinner("Đang phân tích..."):
            processed_text = preprocess(input_text)
            inputs = tokenizer(processed_text, return_tensors="pt", truncation=True, max_length=256)
            with torch.no_grad():
                outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            pred_idx = torch.argmax(probs).item()
            confidence = probs[0][pred_idx].item()

            st.success(f"Kết quả: **{id2label[pred_idx]}**")
            st.info(f"Độ tin cậy: {confidence*100:.2f}%")
            with st.expander("Chi tiết xử lý"):
                st.code(processed_text)
"""

# Ghi nội dung vào file app.py
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

print(" Đã tạo file app.py thành công!")

 Đã tạo file app.py thành công!


<>:34: SyntaxWarning: invalid escape sequence '\S'
<>:34: SyntaxWarning: invalid escape sequence '\S'
/tmp/ipython-input-67298072.py:34: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub(r'http\S+', '', text)


In [3]:
import subprocess
import time

!streamlit run app.py &>/content/logs.txt &
time.sleep(5)

print(" Đang tạo đường hầm Cloudflare...")
!nohup ./cloudflared-linux-amd64 tunnel --url http://localhost:8501 > tunnel.log 2>&1 &
time.sleep(8)


!grep -o 'https://.*\.trycloudflare.com' tunnel.log | head -n 1


 Đang tạo đường hầm Cloudflare...
https://thick-verbal-interests-advocacy.trycloudflare.com
